In [1]:
import pandas as pd
import requests
import ast
import mariadb
import sys
from datetime import datetime, timedelta

In [2]:
dt_today = datetime.now()
dt_tomorrow = datetime.now() + timedelta(days=1)
str_date_today = dt_today.strftime("%m-%d-%Y")
str_date_tomorrow = dt_tomorrow.strftime("%m-%d-%Y")

In [3]:
URL = f"https://www.albion-online-data.com/api/v2/stats/gold?date={str_date_today}&end_date={str_date_tomorrow}"
URL

'https://www.albion-online-data.com/api/v2/stats/gold?date=08-24-2022&end_date=08-25-2022'

In [4]:
try:
    conn = mariadb.connect(
        user="ciurlini",
        password="casa2406",
        host="localhost",
        port=3306,
        database="ALBION"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [27]:
response = requests.get(URL)
content = response.content
content_string = content.decode('UTF-8')
content_list = ast.literal_eval(content_string)

In [20]:
df_index = pd.DataFrame(content_list)
df_index = df_index[['timestamp', 'price']]
df_index[['timestamp']] = df_index[['timestamp']].apply(pd.to_datetime)

In [21]:
df_dbmaria = pd.read_sql(
    "SELECT  * FROM ALBION.gold_prices", conn)

c:\Users\victo.000\Documents\python\gold_predict\.pyenv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
df_diff = pd.concat([df_dbmaria,df_index]).drop_duplicates(keep=False)

In [23]:
df_diff

,timestamp,price
0,2020-08-23 00:00:00,2796
1,2020-08-23 01:00:00,2799
2,2020-08-23 02:00:00,2797
3,2020-08-23 03:00:00,2795
4,2020-08-23 04:00:00,2796
...,...,...
16608,2022-08-23 19:00:00,3428
16609,2022-08-23 20:00:00,3448
16610,2022-08-23 21:00:00,3428
16611,2022-08-23 22:00:00,3445


In [24]:
df_ingest = df_diff.astype({'timestamp': str, 'price': int})

In [25]:
cols = "`,`".join([str(i) for i in df_ingest.columns.tolist()])

In [26]:
for i,row in df_ingest.iterrows():
    sql = "INSERT INTO `gold_prices` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))
conn.commit()

In [28]:
len(df_index)

16628

In [32]:
list_of_cities = ['Bridgewatch', 'Caerleon', 'Lymhurst', 'Martlock', 'Thetford']

In [33]:
cities = ','.join(list_of_cities)

In [36]:
URL_ITEMS = f"https://www.albion-online-data.com/api/v2/stats/history/T4_BAG?date=8-24-2022&end_date=8-25-2022&locations={cities}&qualities=1,2,3&time-scale=6"

In [37]:
URL_ITEMS

'https://www.albion-online-data.com/api/v2/stats/history/T4_BAG?date=8-24-2022&end_date=8-25-2022&locations=Bridgewatch,Caerleon,Lymhurst,Martlock,Thetford&qualities=1,2,3&time-scale=6'